In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

In [5]:
# Load and preprocess your data
sales_data_path = '../data/supermarket_sales.csv'  # Update with your actual path
sales_data = pd.read_csv(sales_data_path)
sales_data['Date'] = pd.to_datetime(sales_data['Date'], format='%m/%d/%Y')

In [6]:
# Group by branch and date, and aggregate the total sales
branch_sales = sales_data.groupby(['Branch', 'Date']).agg({'Total': 'sum'}).reset_index()

In [7]:
# Function to create lag features and train/test sets for each branch
def create_branch_data(branch_data):
    for lag in range(1, 8):
        branch_data[f'lag_{lag}'] = branch_data['Total'].shift(lag)
    branch_data = branch_data.dropna()

    X = branch_data[[f'lag_{i}' for i in range(1, 8)]]
    y = branch_data['Total']
    return X, y

In [8]:
# Separate data by branch
branches = ['A', 'B', 'C']
mlp_predictions = {}
mae_results = {}
accuracy_results = {}

In [9]:
for branch in branches:
    branch_data = branch_sales[branch_sales['Branch'] == branch].copy()
    X, y = create_branch_data(branch_data)
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Normalize the data
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train the MLP model
    mlp_model = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=42)
    mlp_model.fit(X_train_scaled, y_train)

    # Predict the next 7 sales
    predictions = mlp_model.predict(X_test_scaled[:7])

    # Calculate MAE and Accuracy
    mae = mean_absolute_error(y_test[:7], predictions)
    accuracy = 100 - (mae / y_test[:7].mean() * 100)

    # Store results
    mlp_predictions[branch] = predictions
    mae_results[branch] = mae
    accuracy_results[branch] = accuracy

c:\Users\renan\source\repos\mlp-sales-prediction\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\renan\source\repos\mlp-sales-prediction\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\renan\source\repos\mlp-sales-prediction\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [10]:
# Display results
for branch in branches:
    print(f"Branch {branch}: {mlp_predictions[branch]} MAE: {mae_results[branch]} Accuracy: {accuracy_results[branch]}%")

Branch A: [1398.24791205 1300.97848998 1422.23891774 1007.93877099 1160.73591644
 1158.72356139  959.45151597] MAE: 633.1510805367004 Accuracy: 51.961295920893875%
Branch B: [1108.31033991 1289.83546976 1195.19592629 1331.964008   1258.72586571
  976.01183082 1091.21418871] MAE: 709.2004696765441 Accuracy: 17.057815099801417%
Branch C: [1074.12258015 1225.6944691  1284.08805733 1598.10833739 1239.35260262
 1402.44154669 1139.14599079] MAE: 1059.6072377548237 Accuracy: 27.363017378682216%
